In [115]:
import os
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor

import warnings
warnings.filterwarnings('ignore')


In [116]:
data_csv = os.path.join('data', 'final_merged_data.csv')
df = pd.read_csv(data_csv)

In [117]:
# We have 115 stations in total

unique_stations = df['station_id'].unique()
len(list(df["station_id"].value_counts()))

115

#### Select only the columns we want

In [118]:
df['air_temperature_celsius'] = (df['max_air_temperature_celsius'] + df['min_air_temperature_celsius']) / 2
df['relative_humidity_percent'] = (df['max_relative_humidity_percent'] + df['min_relative_humidity_percent']) / 2


cols_to_keep = [
    'station_id', # Model name
    'num_bikes_available', # Precited value
    'last_reported', # Get the day of the week
    'hour', 'minute',
    'air_temperature_celsius', 
    'relative_humidity_percent'
]

## 1st Model test

- `time_of_day` represents the proportion of the day that has passed.
- `time_sin` and `time_cos` capture the cyclic pattern of the day.

In [ ]:
df_testing_models = df[cols_to_keep].copy()

# Get the day of the week
df_testing_models['last_reported'] = pd.to_datetime(df_testing_models['last_reported'])
day_codes = df_testing_models['last_reported'].dt.dayofweek
df_testing_models['day_of_week'] = pd.Categorical.from_codes(day_codes, categories=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])

# Combine hour and minute into a single variable: fraction of the day
df_testing_models['time_of_day'] = (df_testing_models['hour'] * 60 + df_testing_models['minute']) / (24 * 60)

# Create cyclical features from the time_of_day (range 0-1)
df_testing_models['time_sin'] = np.sin(2 * np.pi * df_testing_models['time_of_day'])
df_testing_models['time_cos'] = np.cos(2 * np.pi * df_testing_models['time_of_day'])

# One-hot encode the day_of_week categorical feature
df_testing_models = pd.get_dummies(df_testing_models, columns=['day_of_week'], drop_first=True)

# drop
df_testing_models = df_testing_models.drop(columns=['last_reported'])
df_testing_models = df_testing_models.drop(columns=['hour', 'minute', 'time_of_day'])

In [ ]:
df_testing_models.head()

,station_id,num_bikes_available,air_temperature_celsius,relative_humidity_percent,time_sin,time_cos,day_of_week_Tue,day_of_week_Wed,day_of_week_Thu,day_of_week_Fri,day_of_week_Sat,day_of_week_Sun
0,10,15,13.955,83.75,0.043619,0.999048,False,False,False,False,False,True
1,100,17,13.955,83.75,0.043619,0.999048,False,False,False,False,False,True
2,109,20,13.955,83.75,0.043619,0.999048,False,False,False,False,False,True
3,11,1,13.955,83.75,0.043619,0.999048,False,False,False,False,False,True
4,114,4,13.955,83.75,0.043619,0.999048,False,False,False,False,False,True


#### Compare the performce of these 4 models:

- Linear Regression
- Random Forest
- Gradient Boosting
- XgBoost

In [127]:
def train_model(df_model, station_id):
    """
    Train four models (Linear Regression, Random Forest, Gradient Boosting, XGBoost)
    for a given station dataframe, evaluate them using RMSE, save the results to a .txt file,
    and save the models in pickle files in a station-specific folder inside 'model_outputs'.
    """
    # Create a folder for this station inside 'model_outputs'
    base_output_folder = 'model_outputs'
    station_folder = os.path.join(base_output_folder, f"station_{station_id}")
    os.makedirs(station_folder, exist_ok=True)

    # Define feature columns and the target
    feature_cols = [
        'time_sin', 'time_cos', 
        'air_temperature_celsius',
        'relative_humidity_percent'
    ]
    # Include one-hot encoded day_of_week columns if they exist
    day_cols = [col for col in df_model.columns if col.startswith('day_of_week_')]
    feature_cols += day_cols

    target = 'num_bikes_available'

    # Prepare X and y
    X = df_model[feature_cols]
    y = df_model[target]

    # Split data into training and test sets (20% test)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    results = {}
    # Put a very high RMSE for the station_id, so the next code can work
    results['station_id'] = {'station_id': int(station_id), 'Test_RMSE': 99999}

    # ---------------------------
    # 1. Linear Regression Model
    # ---------------------------
    lr = LinearRegression()
    lr_cv_scores = cross_val_score(lr, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    lr_cv_rmse = np.sqrt(-lr_cv_scores.mean())
    
    lr.fit(X_train, y_train)
    y_pred_lr = lr.predict(X_test)
    lr_rmse = np.sqrt(mean_squared_error(y_test, y_pred_lr))
    results['Linear Regression'] = {'CV_RMSE': lr_cv_rmse, 'Test_RMSE': lr_rmse}

    # Save the Linear Regression model
    lr_filename = os.path.join(station_folder, f"model_station_{station_id}_LinearReg.pkl")
    with open(lr_filename, 'wb') as f:
        pickle.dump(lr, f)

    # ---------------------------
    # 2. Random Forest Regressor
    # ---------------------------
    rf = RandomForestRegressor(random_state=42)
    rf_param_grid = {
        'n_estimators': [50, 100],
        'max_depth': [None, 5, 10],
        'min_samples_split': [2, 5]
    }
    rf_grid = GridSearchCV(rf, rf_param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    rf_grid.fit(X_train, y_train)
    rf_best = rf_grid.best_estimator_
    
    y_pred_rf = rf_best.predict(X_test)
    rf_rmse = np.sqrt(mean_squared_error(y_test, y_pred_rf))
    results['Random Forest'] = {
        'Best_Params': rf_grid.best_params_,
        'Test_RMSE': rf_rmse
    }

    # Save the Random Forest model
    rf_filename = os.path.join(station_folder, f"model_station_{station_id}_RandomForest.pkl")
    with open(rf_filename, 'wb') as f:
        pickle.dump(rf_best, f)

    # ---------------------------
    # 3. Gradient Boosting Regressor
    # ---------------------------
    gbr = GradientBoostingRegressor(random_state=42)
    gbr_param_grid = {
        'n_estimators': [50, 100],
        'learning_rate': [0.05, 0.1, 0.2],
        'max_depth': [3, 5]
    }
    gbr_grid = GridSearchCV(gbr, gbr_param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    gbr_grid.fit(X_train, y_train)
    gbr_best = gbr_grid.best_estimator_
    
    y_pred_gbr = gbr_best.predict(X_test)
    gbr_rmse = np.sqrt(mean_squared_error(y_test, y_pred_gbr))
    results['Gradient Boosting'] = {
        'Best_Params': gbr_grid.best_params_,
        'Test_RMSE': gbr_rmse
    }

    # Save the Gradient Boosting model
    gbr_filename = os.path.join(station_folder, f"model_station_{station_id}_GBR.pkl")
    with open(gbr_filename, 'wb') as f:
        pickle.dump(gbr_best, f)

    # ---------------------------
    # 4. XGBoost Regressor
    # ---------------------------
    xgb = XGBRegressor(random_state=42, objective='reg:squarederror', verbosity=0)
    xgb_param_grid = {
        'n_estimators': [50, 100],
        'learning_rate': [0.05, 0.1, 0.2],
        'max_depth': [3, 5]
    }
    xgb_grid = GridSearchCV(xgb, xgb_param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    xgb_grid.fit(X_train, y_train)
    xgb_best = xgb_grid.best_estimator_
    
    y_pred_xgb = xgb_best.predict(X_test)
    xgb_rmse = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
    results['XGBoost'] = {
        'Best_Params': xgb_grid.best_params_,
        'Test_RMSE': xgb_rmse
    }

    # Save the XGBoost model
    xgb_filename = os.path.join(station_folder, f"model_station_{station_id}_XGB.pkl")
    with open(xgb_filename, 'wb') as f:
        pickle.dump(xgb_best, f)

    # ---------------------------
    # Save results to a .txt file
    # ---------------------------
    results_filename = os.path.join(station_folder, f"results_station_{station_id}.txt")
    with open(results_filename, 'w') as f:
        f.write(f"Results for station {station_id}\n\n")
        f.write("Linear Regression:\n")
        f.write(f"  CV RMSE: {results['Linear Regression']['CV_RMSE']:.2f}\n")
        f.write(f"  Test RMSE: {results['Linear Regression']['Test_RMSE']:.2f}\n\n")
        
        f.write("Random Forest:\n")
        f.write(f"  Best Parameters: {results['Random Forest']['Best_Params']}\n")
        f.write(f"  Test RMSE: {results['Random Forest']['Test_RMSE']:.2f}\n\n")
        
        f.write("Gradient Boosting:\n")
        f.write(f"  Best Parameters: {results['Gradient Boosting']['Best_Params']}\n")
        f.write(f"  Test RMSE: {results['Gradient Boosting']['Test_RMSE']:.2f}\n\n")
        
        f.write("XGBoost:\n")
        f.write(f"  Best Parameters: {results['XGBoost']['Best_Params']}\n")
        f.write(f"  Test RMSE: {results['XGBoost']['Test_RMSE']:.2f}\n")
    
    
    return results



In [ ]:
count_best_linear_reg = 0
count_best_random_forest = 0
count_best_gradient_boosting = 0
count_best_xgboost = 0
test_1_results = {}

station_dfs = {}

for station in unique_stations:
    station_df = df_testing_models[df_testing_models['station_id'] == station].copy()
    station_df = station_df.drop(columns=['station_id'])
    station_df = station_df.reset_index(drop=True)
    station_dfs[station] = station_df


for station_id, station_df in station_dfs.items():
    
    print(f"\nTraining model for station {station_id}")
    
    results = train_model(station_df, station_id)
    test_1_results[int(station_id)] = results
    
    best_model = min(results, key=lambda x: results[x]['Test_RMSE'])
    print(f"Best model for station {station_id}: {best_model}")
    print(f"Test RMSE: {results[best_model]['Test_RMSE']:.2f}")
    
    if best_model == 'Linear Regression':
        count_best_linear_reg += 1
    elif best_model == 'Random Forest':
        count_best_random_forest += 1
    elif best_model == 'Gradient Boosting':
        count_best_gradient_boosting += 1
    elif best_model == 'XGBoost':
        count_best_xgboost += 1
    else:
        print("Error: Unknown model type")


    


Training model for station 10
Best model for station 10: Random Forest
Test RMSE: 1.10

Training model for station 100
Best model for station 100: Random Forest
Test RMSE: 1.98

Training model for station 109
Best model for station 109: Random Forest
Test RMSE: 2.36

Training model for station 11
Best model for station 11: Random Forest
Test RMSE: 2.95

Training model for station 114
Best model for station 114: Random Forest
Test RMSE: 3.48

Training model for station 116
Best model for station 116: Random Forest
Test RMSE: 2.67

Training model for station 13
Best model for station 13: Random Forest
Test RMSE: 2.31

Training model for station 14
Best model for station 14: Random Forest
Test RMSE: 3.25

Training model for station 15
Best model for station 15: Random Forest
Test RMSE: 0.68

Training model for station 17
Best model for station 17: Random Forest
Test RMSE: 2.70

Training model for station 18
Best model for station 18: Random Forest
Test RMSE: 2.96

Training model for stat

### Test 1 - Results

In [ ]:
print(f"\nSummary of best models across all stations:")
print(f"Linear Regression: {count_best_linear_reg}")
print(f"Random Forest: {count_best_random_forest}")
print(f"Gradient Boosting: {count_best_gradient_boosting}")
print(f"XGBoost: {count_best_xgboost}")


Summary of best models across all stations:
Linear Regression: 2
Random Forest: 110
Gradient Boosting: 2
XGBoost: 1


- Almost all have Random Forest as Best Estimator (110/115)
- Let's check what happens in the other 5 cases, if the distance between the best and Random Forest is low we keep that one

In [138]:
for station_id, result in test_1_results.items():
    
    result_Random_Forest = result['Random Forest']['Test_RMSE']
    result_min = min(result['Gradient Boosting']['Test_RMSE'], result['XGBoost']['Test_RMSE'], result['Linear Regression']['Test_RMSE'])
    
    if result_min < result_Random_Forest:
        
        best_model = min(results, key=lambda x: results[x]['Test_RMSE'])
        print(f"\nBest model for station {station_id} --> {best_model}: {result_min}")
        print(f"Test RMSE for RandomForest: {result['Random Forest']['Test_RMSE']:.2f}")
        print(f"Number of Observations: {len(station_dfs[station_id])}")
            


Best model for station 20 --> Linear Regression: 2.183464289397626
Test RMSE for RandomForest: 2.19
Number of Observations: 2693

Best model for station 101 --> Linear Regression: 1.5906287771961192
Test RMSE for RandomForest: 1.65
Number of Observations: 2311

Best model for station 104 --> Linear Regression: 3.640122669584307
Test RMSE for RandomForest: 3.70
Number of Observations: 2464


- We only get 3 cases, out of the 5.
- Plus all 3 are Linear Regression when supoosedly there should only be 2 cases of it being the best one

> This is probbaly due to such a small rounding number difference, that we can keep RandomForest as the best model for all stations

## Now chose the best parameters

- Grid Search over many parameters, and keep the model with the minimum RMSE

In [ ]:

def train_rf_model_for_station(df_model, station_id):
    """
    Train a Random Forest Regressor using an expanded grid search for a given station dataframe.
    Save only the best model and write a text file with the results, including which model was kept.
    The outputs are saved in a folder structure 'model_outputs/{station_id}'.
    """
    # Create a folder for the specific station inside 'model_outputs'
    base_output_folder = 'model_outputs_V2'
    station_folder = os.path.join(base_output_folder, f"{station_id}")
    os.makedirs(station_folder, exist_ok=True)

    # Define feature columns and the target
    feature_cols = [
        'time_sin', 'time_cos', 
        'air_temperature_celsius',
        'relative_humidity_percent'
        
    ]
    # Add one-hot encoded day_of_week columns if available
    day_cols = [col for col in df_model.columns if col.startswith('day_of_week_')]
    feature_cols += day_cols

    target = 'num_bikes_available'

    # Prepare X and y
    X = df_model[feature_cols]
    y = df_model[target]

    # Split data into training and test sets (20% test)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # ---------------------------
    # Random Forest Regressor with Expanded Grid Search
    # ---------------------------
    rf = RandomForestRegressor(random_state=42)
    rf_param_grid = {
        'n_estimators': [50, 100, 200], 
        'max_depth': [None, 5, 10, 15],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    rf_grid = GridSearchCV(rf, rf_param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    rf_grid.fit(X_train, y_train)
    rf_best = rf_grid.best_estimator_
    
    # Evaluate on the test set
    y_pred_rf = rf_best.predict(X_test)
    rf_rmse = np.sqrt(mean_squared_error(y_test, y_pred_rf))
    
    # Save the best Random Forest model
    rf_filename = os.path.join(station_folder, f"model_station_{station_id}_RandomForest.pkl")
    with open(rf_filename, 'wb') as f:
        pickle.dump(rf_best, f)
    
    # Save results to a .txt file
    results_filename = os.path.join(station_folder, f"results_station_{station_id}.txt")
    with open(results_filename, 'w') as f:
        f.write(f"Results for station {station_id}\n")
        f.write("Kept Model: Random Forest Regressor\n")
        f.write("Best Parameters: " + str(rf_grid.best_params_) + "\n")
        f.write(f"Test RMSE: {rf_rmse:.2f}\n")
    
    print(f"Station {station_id} best model saved in '{station_folder}'.")
    print(f"Best Parameters: {rf_grid.best_params_}")
    print(f"Test RMSE: {rf_rmse:.2f}")

# Example: Training for each station if station_dfs is a dictionary of station_id -> station DataFrame.
for station_id, station_df in station_dfs.items():
    print(f"\nTraining Random Forest model for station {station_id}")
    train_rf_model_for_station(station_df, station_id)



Training Random Forest model for station 10
Station 10 best model saved in 'model_outputs_V2/10'.
Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Test RMSE: 1.10

Training Random Forest model for station 100
Station 100 best model saved in 'model_outputs_V2/100'.
Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Test RMSE: 1.98

Training Random Forest model for station 109
Station 109 best model saved in 'model_outputs_V2/109'.
Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Test RMSE: 2.40

Training Random Forest model for station 11
Station 11 best model saved in 'model_outputs_V2/11'.
Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Test RMSE: 2.95

Training Random Forest model for station 114
Station 114 best model saved in 'model_outputs_V2/114'.
Best Paramet

In [ ]:
import os
import shutil

# Define the input folder where the station subfolders are stored and the output folder for aggregated models
input_folder = 'model_outputs_v2'
output_folder = 'pickle_models'
os.makedirs(output_folder, exist_ok=True)

# Loop through each subfolder in 'model_outputs'
for station_folder in os.listdir(input_folder):
    station_path = os.path.join(input_folder, station_folder)
    if os.path.isdir(station_path):
        # Loop through each file in the station's folder
        for filename in os.listdir(station_path):
            # Look for pickle files that follow the naming convention
            if filename.startswith("model_station_") and filename.endswith(".pkl"):
                src = os.path.join(station_path, filename)
                # Extract the station id from the file name.
                # Expected format: "model_station_{station_id}_RandomForest.pkl"
                parts = filename.split('_')
                if len(parts) >= 3:
                    station_id_extracted = parts[2]
                    new_filename = f"model_station_{station_id_extracted}.pkl"
                else:
                    new_filename = filename
                dst = os.path.join(output_folder, new_filename)
                shutil.copy(src, dst)
                print(f"Copied {src} to {dst}")


Copied model_outputs_v2/61/model_station_61_RandomForest.pkl to pickle_models/model_station_61.pkl
Copied model_outputs_v2/95/model_station_95_RandomForest.pkl to pickle_models/model_station_95.pkl
Copied model_outputs_v2/59/model_station_59_RandomForest.pkl to pickle_models/model_station_59.pkl
Copied model_outputs_v2/92/model_station_92_RandomForest.pkl to pickle_models/model_station_92.pkl
Copied model_outputs_v2/66/model_station_66_RandomForest.pkl to pickle_models/model_station_66.pkl
Copied model_outputs_v2/104/model_station_104_RandomForest.pkl to pickle_models/model_station_104.pkl
Copied model_outputs_v2/50/model_station_50_RandomForest.pkl to pickle_models/model_station_50.pkl
Copied model_outputs_v2/68/model_station_68_RandomForest.pkl to pickle_models/model_station_68.pkl
Copied model_outputs_v2/103/model_station_103_RandomForest.pkl to pickle_models/model_station_103.pkl
Copied model_outputs_v2/57/model_station_57_RandomForest.pkl to pickle_models/model_station_57.pkl
Copi